###Package

In [1]:
!pip install langdetect

In [2]:
!pip install googletrans==4.0.0-rc1

In [3]:
!pip install Sastrawi

###Library

In [4]:
import pandas as pd
import re
import unicodedata
import string
from langdetect import detect
from googletrans import Translator
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from langdetect import detect, DetectorFactory
from googletrans import Translator

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###Load dataset

In [6]:
df_comments = pd.read_csv('/content/drive/MyDrive/PemrosesanTeksTeori/Scraping_YT_histID.csv')
df_comments.head()

,comment_ori
0,Pemerintah kita tahun ini ngumumin adanya penu...
1,"jangan lupa bang, kasus di mana kaum mayoritas..."
2,"dulu ga zaman digital, susah nyimpen bukti. ha..."
3,Sekarang sudah diangkat menjadi pahlawan Nasional
4,Sejarah ditulis oleh pemenang


In [7]:
len(df_comments)

1026

###Inisialisasi stopword, stemmer, lemmatizer

In [8]:
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer_en = WordNetLemmatizer()
stopwords_en = set(stopwords.words("english"))

factory = StemmerFactory()
stemmer_id = factory.create_stemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
LEMMA_DICT = {"intah": "perintah", "ontak": "berontak", "pemerintah": "pemerintah", "pemerintahan": "pemerintah", "nyindir": "sindir",
              "nyimpen": "simpen", "nyimpan": "simpan", "nyontek": "sontek", "nyari": "cari", "ngomong": "omong", "sibapak": "bapak",
              "ngegas": "gas", "ngelakuin": "laku", "ngelaku": "laku", "ngebantai": "bantai", "ngumumin": "umum", "petrusnya": "petrus",
              "alasan": "alasan", "ilangin": "hilang", "dibohongin": "bohong", "ajarin": "ajar", "gedein": "gede", "kecilin": "kecil",
              "kedengeran": "dengar", "nyadarin": "sadar", "haluan": "halu", "dipoles": "poles", "pantesan": "pantas", "mendownload":
              "download", "diturunin": "turun", "jilatin": "jilat", "dipajakin": "pajak", "disembunyiin": "sembunyi", "diterapin": "terap",
              "keinget": "ingat", "lainnya": "lain", "bayangin": "bayang", "kelihatanya": "lihat", "membrontak": "brontak", "ngahapus": "hapus",
              "ngakalin": "akal", "direspon": "respon", "diperbaiki": "perbaiki", "disebutin": "sebut", "ngebahas": "bahas", "ngurusin": "urus",
              "pemenang": "menang", "pemenangnya": "menang"

}

STOPWORDS_ID = {
    "yang","sama","jadi","sudah","akan","kita","kalian","mereka","tapi","bahkan","bisa","coba","mungkin",
    "saya","kamu","banget","sangat","lebih","bukan","dari","pada","terlalu","sampai","tersebut","skrng",
    "orang","karena","kalau","untuk","dengan","dong","jangan","mana","juga","makasih","melakukan","videonya",
    "kayak","emang","memang","siapa","bagaimana","bagaiamana","soalnya","saja","atas","ingat","bjir","beuhh",
    "cuma","cuman","masih","gimana","lalu","kali","gini","sampe","biar","bikin","kebanyakan","terimakasih",
    "malah","tahu","makin","adalah","selalu","memiliki","paling","sering","dalam","bang","dilakukan","tiati",
    "hingga","ngak","terus","harus","membuat","adanya","sekali","semakin","melihat","gapernah","nihh","knpa",
    "gitu","bakal","telah","sungguh","menjadi","harap","tetapi","ingin","wkwkwk","wkwk","kdng","kyak","nnti",
    "tbtb","pnya","kontll","sbagai","aowkwkwkwokw","dpet","haha","kalaupun","brooo","a0akah","tidakmtakut",
    "meng","pdhl","sjarah","aduhh","wihh","mmng","adasih","nyernag","bruhh","bnget","kalo","huhu","lahh","adaa",
    "kenapasih","nggk","alamakk","alamak","makanya","hihihi","hadehh","wkwkwkwkwkwk","ituhh","ygkek","donk"
}

###Fungsi prefix

In [10]:
def fix_indonesian_prefix(word):

    # 1. Tangani "ny" prefix informal (ny + vokal / konsonan)
    if word.startswith("ny"):
        # ny + vokal (nyanyian -> anyian, nyeri -> eri)
        if len(word) > 2 and word[2] in "aiueo":
            return word[2:]

        # ny + konsonan (nyindir -> sindir)
        if len(word) > 2:
            return "s" + word[2:]

    # 2. Tangani "nge" prefix (ngebahas -> bahas)
    if word.startswith("nge"):
        if len(word) > 3:
            return word[3:]

    # 3. Tangani "ng" prefix umum (ngurus, ngomong)
    if word.startswith("ng"):
        if len(word) > 2:
            return word[2:]

    # 4. Tangani "di-...-in" (dibohongin, disembunyiin)
    if word.startswith("di") and word.endswith("in"):
        core = word[2:-2]
        return core

    # 5. Tangani "ke-...-in/kan" informal
    if word.startswith("ke") and (word.endswith("in") or word.endswith("kan")):
        return word[2:]

    # 6. Tangani "mem-" / "men-" / "meng-" / "meny-"
    if word.startswith(("mem", "men", "meng", "meny")):
        if len(word) > 4:
            return word[3:]  # buang mem- / men-
        if word.startswith("meng") and len(word) > 4:
            return word[4:]

    return word


###Stemming + lemmatization

In [11]:
def indonesian_stem_lemma(word):
    # 1. Sastrawi stem
    stemmed = stemmer_id.stem(word)

    # 2. Jika ada dalam kamus lemma → override
    if stemmed in LEMMA_DICT:
        return LEMMA_DICT[stemmed]

    # 3. Perbaikan prefix informal
    prefix_fixed = fix_indonesian_prefix(stemmed)

    # 4. Prefix hasil setelah diperbaiki masuk kamus
    if prefix_fixed in LEMMA_DICT:
        final_word = LEMMA_DICT[prefix_fixed]

    # 5. Jika kata original ada di kamus lemma
    if word in LEMMA_DICT:
        return LEMMA_DICT[word]

    return prefix_fixed


###Lemmatization Bahasa Inggris

In [12]:
def english_fallback(tokens):
    cleaned = []
    for w in tokens:
        if w not in stopwords_en:
            w = lemmatizer_en.lemmatize(w)
            cleaned.append(w)
    return cleaned


###Fungsi translate

In [13]:
def translate_to_indonesian(text):
    try:
        lang = detect_language(text)
        if lang == 'en':
            return translate_english_to_indonesian(text).lower()
        else:
            return text.lower()
    except:
        return text.lower()

###Preprocessing

In [14]:
def reduce_repeated_letters(text):
    return re.sub(r'(.)\1{2,}', r'\1\1', text)

In [16]:
def preprocess_text(text):
    if not isinstance(text, str) or len(text.strip()) == 0:
        return ""

    # 2. Lowercase
    text = text.lower()

    # 3. Clean URL, mentions, emoji, punctuation
    text = re.sub(r"http\S+|www\S+", " ", text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r"\s+", " ", text).strip()
    text = translate_to_indonesian(text)
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"[^a-zA-Z\s]", " ", text)
    text = reduce_repeated_letters(text)
    text = text.lower().strip()

    if len(text) == 0:
        return ""

    # 4. Tokenizing
    tokens = text.split()

    # 5. Stopwords
    tokens = [w for w in tokens if w not in STOPWORDS_ID]

    if len(tokens) == 0:
        return ""

    tokens = [w for w in tokens if len(w) > 3]

    # 6. Stemming + Lemmatization Indonesia
    processed = []
    for w in tokens:
        new_w = indonesian_stem_lemma(w)
        processed.append(new_w)

    # 7. Fallback English lemmatization (jaga-jaga)
    processed = english_fallback(processed)

    # 8. Gabungkan kembali
    final_text = " ".join(processed).strip()

    return final_text

# hapus duplikat komentar
df_comments = df_comments.dropna(subset=["comment_ori"])
before = len(df_comments)
df_comments = df_comments.drop_duplicates(subset=["comment_ori"]).reset_index(drop=True)
after = len(df_comments)

df_comments["comment_clean"] = df_comments["comment_ori"].apply(preprocess_text)
df_comments.head()


,comment_ori,comment_clean
0,Pemerintah kita tahun ini ngumumin adanya penu...,pemerintah tahun umum tulis ulang buku sejarah...
1,"jangan lupa bang, kasus di mana kaum mayoritas...",lupa kasus kaum mayoritas negeri bantai kaum m...
2,"dulu ga zaman digital, susah nyimpen bukti. ha...",dulu zaman digital susah simpen bukti hanya li...
3,Sekarang sudah diangkat menjadi pahlawan Nasional,sekarang angkat pahlawan nasional
4,Sejarah ditulis oleh pemenang,sejarah tulis oleh menang


In [17]:
# eksport dataset
df_comments.to_csv('afterPRE.csv', index=False, encoding='utf-8')

###package

In [18]:
!pip install gensim

###Library klasifikasi

In [19]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from gensim.models import Word2Vec
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


###Load data

In [20]:
df = pd.read_csv("/content/drive/MyDrive/PemrosesanTeksTeori/afterPRE.csv")
col_clean = "comment_clean"
df = df.dropna(subset=[col_clean]).reset_index(drop=True)

print("Total data:", len(df))

Total data: 982


###Labeling lexicon

In [21]:
positive_words = [
    "keren", "bagus", "mantap", "mantep", "terbaik", "wow", "suka", "love", "semangat", "edukasi", "edukatif", "paham", "syukur", "baik"
    "bagus banget", "polisi", "luar biasa", "mantap banget", "bagus sekali", "terima kasih", "respect", "hormat", "cerdas", "algoritma",
    "huraa", "cemas", "malu", "plis", "respect", "aparat", "piece", "kolonialisme", "ilmu", "pulih", "supersemar", "wawas", "wawasan", "pasti",
    "sukses", "anime", "favoritt", "ayo", "menang", "nyala"
]

negative_words = [
    "jelek", "buruk", "parah", "kesal", "anjing", "bohong", "rusak", "bobrok", "bohong", "hapus", "bodoh", "gelap",
    "normal", "benci", "kecewa", "hancur", "tidak suka", "nyata", "pejabat", "khianat", "sedih", "miris", "ancam",
    "bahaya", "kejam", "bahlil", "petrus", "propaganda", "brengsek", "prabowo", "seleweng", "dongo", "korup", "korupsi",
    "zionis", "israel", "isrewel"
]

neutral_words = [
    "sejarah", "kobi", "buku", "pelajaran", "menang", "fakta"
    "pemerintah", "absen", "pertama", "masa", "sekolah", "guru"
]

def lexicon_label(text):
    t = text.lower()
    pos = sum(w in t for w in positive_words)
    neg = sum(w in t for w in negative_words)
    neu = sum(w in t for w in neutral_words)

    if pos > neg and pos > neu:
        return "positif"
    elif neg > pos and neg > neu:
        return "negatif"
    elif neu > pos and neu > neg:
        return "netral"
    else:
        return None

df["seed_label"] = df[col_clean].apply(lexicon_label)

pos_seed = df[df["seed_label"] == "positif"].head(100)
neg_seed = df[df["seed_label"] == "negatif"].head(100)
neu_seed = df[df["seed_label"] == "netral"].head(50)

seed_df = pd.concat([pos_seed, neg_seed, neu_seed]).reset_index(drop=True)

print("Seed label counts:")
print(seed_df["seed_label"].value_counts())


Seed label counts:
seed_label
positif    100
negatif    100
netral      50
Name: count, dtype: int64


###Split data

In [22]:
df_filtered = df.dropna(subset=["seed_label"])
X = df_filtered[col_clean]
y = df_filtered["seed_label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

###SVM TF-IDF

In [23]:
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

svm_tfidf = SVC(kernel="linear")
svm_tfidf.fit(X_train_tfidf, y_train)

pred_svm_tfidf = svm_tfidf.predict(X_test_tfidf)

print("\n=== SVM + TF-IDF ===")
print(classification_report(y_test, pred_svm_tfidf))
print("Akurasi:", accuracy_score(y_test, pred_svm_tfidf))



=== SVM + TF-IDF ===
              precision    recall  f1-score   support

     negatif       0.74      0.61      0.67        23
      netral       0.91      0.93      0.92        45
     positif       0.78      0.86      0.82        29

    accuracy                           0.84        97
   macro avg       0.81      0.80      0.80        97
weighted avg       0.83      0.84      0.83        97

Akurasi: 0.8350515463917526


###SVM + WORD2VEC

In [24]:
# Train Word2Vec
sentences = [text.split() for text in X_train]
w2v = Word2Vec(sentences, vector_size=300, window=5, min_count=2, workers=4)

# fungsi rata-rata embedding
def avg_vector(words):
    vecs = [w2v.wv[w] for w in words if w in w2v.wv]
    return np.mean(vecs, axis=0) if len(vecs) > 0 else np.zeros(300)

# buat fitur word2vec
X_train_w2v = np.array([avg_vector(text.split()) for text in X_train])
X_test_w2v = np.array([avg_vector(text.split()) for text in X_test])

In [25]:
# train SVM
svm_w2v = SVC(kernel="linear")
svm_w2v.fit(X_train_w2v, y_train)

pred_svm_w2v = svm_w2v.predict(X_test_w2v)

print("\n=== SVM + Word2Vec ===")
print(classification_report(y_test, pred_svm_w2v))
print("Akurasi:", accuracy_score(y_test, pred_svm_w2v))



=== SVM + Word2Vec ===
              precision    recall  f1-score   support

     negatif       0.00      0.00      0.00        23
      netral       0.46      1.00      0.63        45
     positif       0.00      0.00      0.00        29

    accuracy                           0.46        97
   macro avg       0.15      0.33      0.21        97
weighted avg       0.22      0.46      0.29        97

Akurasi: 0.4639175257731959


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


###LSTM + TF-IDF (Tokenized)

In [26]:
# tokenizer
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

seq_train = tokenizer.texts_to_sequences(X_train)
seq_test = tokenizer.texts_to_sequences(X_test)

maxlen = 50
X_train_pad = pad_sequences(seq_train, maxlen=maxlen)
X_test_pad = pad_sequences(seq_test, maxlen=maxlen)

# encoding
label_map = {"positif":0, "negatif":1, "netral":2}
y_train_id = y_train.replace(label_map)
y_test_id = y_test.replace(label_map)


/tmp/ipython-input-1839953830.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train_id = y_train.replace(label_map)
/tmp/ipython-input-1839953830.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_test_id = y_test.replace(label_map)


In [27]:
# model
model_lstm_tfidf = Sequential([
    Embedding(10000, 128, input_length=maxlen),
    LSTM(128),
    Dropout(0.3),
    Dense(3, activation="softmax")
])

model_lstm_tfidf.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

model_lstm_tfidf.fit(
    X_train_pad, y_train_id,
    epochs=5, batch_size=32, validation_split=0.1
)

pred_lstm_tfidf = model_lstm_tfidf.predict(X_test_pad).argmax(axis=1)

print("\n=== LSTM + TF-IDF Tokenized ===")
print(classification_report(y_test_id, pred_lstm_tfidf))
print("Akurasi:", accuracy_score(y_test_id, pred_lstm_tfidf))

Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 206ms/step - accuracy: 0.3953 - loss: 1.0877 - val_accuracy: 0.6154 - val_loss: 1.0154
Epoch 2/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.5947 - loss: 1.0131 - val_accuracy: 0.6154 - val_loss: 0.8796
Epoch 3/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.6082 - loss: 0.8237 - val_accuracy: 0.6667 - val_loss: 0.7722
Epoch 4/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - accuracy: 0.7441 - loss: 0.6514 - val_accuracy: 0.7179 - val_loss: 0.6784
Epoch 5/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - accuracy: 0.8548 - loss: 0.5053 - val_accuracy: 0.8718 - val_loss: 0.5838
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step

=== LSTM + TF-IDF Tokenized ===
              precision    recall  f1-score   support

           0       0.73      0.93      0.82        29
           1       0.85      0.48      0.61        23
           2       0.85      0.89      0.87        45

    accuracy                           0.80        97
   macro avg       0.81      0.77   

###LSTM + Word2Vec

In [28]:
# index kata
word_index = {word: idx+1 for idx, word in enumerate(w2v.wv.index_to_key)}

# embedding matrix
embedding_matrix = np.zeros((len(word_index) + 1, 300))

for word, idx in word_index.items():
    embedding_matrix[idx] = w2v.wv[word]

# index word2vec
seq_train_w2v = [[word_index.get(w, 0) for w in text.split()] for text in X_train]
seq_test_w2v = [[word_index.get(w, 0) for w in text.split()] for text in X_test]

X_train_w2v_pad = pad_sequences(seq_train_w2v, maxlen=maxlen)
X_test_w2v_pad = pad_sequences(seq_test_w2v, maxlen=maxlen)

In [29]:
# model
model_lstm_w2v = Sequential([
    Embedding(
        input_dim=embedding_matrix.shape[0],
        output_dim=300,
        weights=[embedding_matrix],
        input_length=maxlen,
        trainable=False
    ),
    LSTM(128),
    Dropout(0.3),
    Dense(3, activation="softmax")
])

model_lstm_w2v.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

model_lstm_w2v.fit(
    X_train_w2v_pad, y_train_id,
    epochs=5, batch_size=32, validation_split=0.1
)

pred_lstm_w2v = model_lstm_w2v.predict(X_test_w2v_pad).argmax(axis=1)

print("\n=== LSTM + Word2Vec ===")
print(classification_report(y_test_id, pred_lstm_w2v))
print("Akurasi:", accuracy_score(y_test_id, pred_lstm_w2v))

Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 124ms/step - accuracy: 0.4664 - loss: 1.0910 - val_accuracy: 0.5128 - val_loss: 1.0209
Epoch 2/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - accuracy: 0.4686 - loss: 1.0412 - val_accuracy: 0.5128 - val_loss: 1.0216
Epoch 3/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 161ms/step - accuracy: 0.4698 - loss: 1.0342 - val_accuracy: 0.5128 - val_loss: 1.0050
Epoch 4/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 149ms/step - accuracy: 0.4650 - loss: 1.0147 - val_accuracy: 0.5128 - val_loss: 1.0339
Epoch 5/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 103ms/step - accuracy: 0.5445 - loss: 1.0004 - val_accuracy: 0.6410 - val_loss: 0.9812
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step

=== LSTM + Word2Vec ===
              precision    recall  f1-score   support

           0       0.64      0.62      0.63        29
           1       0.00      0.00      0.00        23
           2       0.62      0.96      0.75        45

    accuracy                           0.63        97
   macro avg       0.42      0.53      0

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
